# RNN Input Data Analysis and Diagonise 

In [1]:
import matplotlib.pyplot as plt

In [2]:

derbaseDir =  ... 
sessionName = 't12.2022.05.05' # 't12.2022.05.03_fiftyWordSet' # 't12.2022.04.28'


import os

if os.path.isdir(derbaseDir):
    print(f"The directory {derbaseDir} exists.")
else:
    print(f"The directory {derbaseDir} does not exist.")

The directory /net2/sandbox/willetspeech_decoder/rnn_cls/input/randTVT_TX1-SP exists.


# Dataset Schema 

In [3]:
import pathlib
import tensorflow as tf
import numpy as np

isTraining = False 

def _loadDataset(fileDir):
    files = sorted([str(x) for x in pathlib.Path(fileDir).glob("*.tfrecord")])
    print(files)
    if isTraining:
        random.shuffle(files)
    dataset = tf.data.TFRecordDataset(files)
    return dataset

nInputFeatures = 256
chanIndices = np.array(range(nInputFeatures))
maxSeqElements = 100 # 100 for phoneme 
datasetFeatures = {
    "inputFeatures": tf.io.FixedLenSequenceFeature([nInputFeatures], tf.float32, allow_missing=True),
    #"classLabelsOneHot": tf.io.FixedLenSequenceFeature([self.nClasses+1], tf.float32, allow_missing=True),
    "newClassSignal": tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    "ceMask": tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    "seqClassIDs": tf.io.FixedLenFeature((maxSeqElements), tf.int64),
    "nTimeSteps": tf.io.FixedLenFeature((), tf.int64),
    "nSeqElements": tf.io.FixedLenFeature((), tf.int64),
    "transcription": tf.io.FixedLenFeature((maxSeqElements), tf.int64)
}

def parseDatasetFunctionSimple(exampleProto):
    dat = tf.io.parse_single_example(exampleProto, datasetFeatures)
    if chanIndices is not None:
        newDat = {}
        newDat['seqClassIDs'] = dat['seqClassIDs']
        newDat['nSeqElements'] = dat['nSeqElements']
        newDat['transcriptions'] = dat['transcription']
        newDat['nTimeSteps'] = dat['nTimeSteps']
        newDat['newClassSignal'] = dat['newClassSignal']
        newDat['ceMask'] = dat['ceMask']
        print(dat['inputFeatures'])
        selectChans = tf.gather(dat['inputFeatures'], tf.constant(chanIndices),axis=-1)
        paddings = [[0, 0], [0, 256-tf.shape(selectChans)[-1]]]
        newDat['inputFeatures'] = tf.pad(selectChans, paddings, 'CONSTANT',constant_values=0)
        print(tf.shape(newDat['inputFeatures']))
        return newDat
    else:
        return dat

2024-09-13 00:44:11.346704: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2024-09-13 00:44:11.346798: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/jih201/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
def get_PHONE_DEF(): 
    PHONE_DEF = [
        'N/A', 'AA', 'AE', 'AH', 'AO', 'AW',
        'AY', 'B',  'CH', 'D', 'DH',
        'EH', 'ER', 'EY', 'F', 'G',
        'HH', 'IH', 'IY', 'JH', 'K',
        'L', 'M', 'N', 'NG', 'OW',
        'OY', 'P', 'R', 'S', 'SH',
        'T', 'TH', 'UH', 'UW', 'V',
        'W', 'Y', 'Z', 'ZH', 'SIL'
    ]
    return PHONE_DEF

PHONE_DEF = get_PHONE_DEF()

In [5]:
# Assuming 0 is the padding value
def remove_padding(tensor, padding_value=0):
    mask = tf.not_equal(tensor, padding_value)  # Create a mask for non-padding values
    cleaned_tensor = tf.boolean_mask(tensor, mask)  # Remove padding based on mask
    return cleaned_tensor

# Load Dataset 
- seqClassIDs -> trueSeqs

In [12]:

formated_data = {} 

for part in ['train', 'test']: 
    tfRecordFolder = derbaseDir+'/'+sessionName+'/'+part+'/'

    formated_data[part] = {} 
    for fn in ['inputFeatures', 'transcriptions', 'trueText', 'trueSeqs', 'seqClassIDs', 'ClassIDs', 'seqStartIdx', 'trueSeqLengths']: 
        formated_data[part][fn] = [] 
    formated_data[part]['nTrials'] = 0
    
    parsed_dataset = _loadDataset(tfRecordFolder).map(parseDatasetFunctionSimple, num_parallel_calls=tf.data.AUTOTUNE)
    formated_data[part]['seqStartIdx'].append(0) 
    for parsed_record in parsed_dataset:
        formated_data[part]['inputFeatures'].append(parsed_record['inputFeatures'])
        tmp = remove_padding(parsed_record['transcriptions'])
        formated_data[part]['transcriptions'].append(tmp)
        formated_data[part]['trueText'].append(''.join([chr(i) for i in tmp]))
        tmp = remove_padding(parsed_record['seqClassIDs'])
        formated_data[part]['seqClassIDs'].append(tmp)
        formated_data[part]['trueSeqs'].append(parsed_record['seqClassIDs'])
        formated_data[part]['ClassIDs'].extend([PHONE_DEF[i] for i in tmp])
        formated_data[part]['seqStartIdx'].append(formated_data[part]['seqStartIdx'][-1]+len(tmp))
        formated_data[part]['nTrials']+=1
        formated_data[part]['trueSeqLengths'].append(parsed_record['nSeqElements'].numpy())

    
    for field in ['trueSeqs']:
        formated_data[part][field] = np.array(formated_data[part][field])

['/net2/sandbox/willetspeech_decoder/rnn_cls/input/randTVT_TX1-SP/t12.2022.05.05/train/chunk_0.tfrecord']
Tensor("ParseSingleExample/ParseExample/ParseExampleV2:1", shape=(None, 256), dtype=float32)
Tensor("Shape_1:0", shape=(2,), dtype=int32)
['/net2/sandbox/willetspeech_decoder/rnn_cls/input/randTVT_TX1-SP/t12.2022.05.05/val/chunk_0.tfrecord']
Tensor("ParseSingleExample/ParseExample/ParseExampleV2:1", shape=(None, 256), dtype=float32)
Tensor("Shape_1:0", shape=(2,), dtype=int32)
['/net2/sandbox/willetspeech_decoder/rnn_cls/input/randTVT_TX1-SP/t12.2022.05.05/test/chunk_0.tfrecord']
Tensor("ParseSingleExample/ParseExample/ParseExampleV2:1", shape=(None, 256), dtype=float32)
Tensor("Shape_1:0", shape=(2,), dtype=int32)


### Data Dict `formted_data` Structure 

In [ ]:
formated_data['train'].keys()

for key in formated_data['train'].keys(): 
    try: 
        print(f"{key} : {np.shape(formated_data['train'][key])}")
    except:
        print(key)

### Example Transcription Sentence 

In [8]:
for split in formated_data.keys(): 
    print(f"{split}: {len(formated_data[split]['transcriptions'])} sentences")
    for i in range(5):
        text = ''.join(chr(value) for value in formated_data[split]['transcriptions'][i] if value!=0 )
        print(f' - {text}')
    print('\n')

train: 324 sentences
 - december and january are nice months to spend in miami
 - vietnamese cuisine is exquisite
 - remove the splinter with a pair of tweezers
 - be careful not to plow over the flower beds
 - well now we have two big theaters


val: 36 sentences
 - bob bandaged both wounds with the skill of a doctor
 - the diagnosis was discouraging however he was not overly worried
 - why charge money for such garbage
 - ambidextrous pickpockets accomplish more
 - gregory and tom chose to watch cartoons in the afternoon


test: 20 sentences
 - to some extent predispositions are shaped by exposure to group environments
 - an adult male baboon's teeth are not suitable for eating shellfish
 - in this context it would do well for us to bear in mind the vision of peace
 - you're boiling milk ain't you
 - rich looked for spotted hyenas and jaguars on the safari


